In [9]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [10]:

df = pd.read_csv("exoplanet_data.csv")
df = df.dropna(axis='columns', how='all')
df = df.dropna()
df.head()

koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  \
0       CONFIRMED              0              0              0              0   
1  FALSE POSITIVE              0              1              0              0   
2  FALSE POSITIVE              0              1              0              0   
3       CONFIRMED              0              0              0              0   
4       CONFIRMED              0              0              0              0   

   koi_period  koi_period_err1  koi_period_err2  koi_time0bk  \
0   54.418383     2.479000e-04    -2.479000e-04   162.513840   
1   19.899140     1.490000e-05    -1.490000e-05   175.850252   
2    1.736952     2.630000e-07    -2.630000e-07   170.307565   
3    2.525592     3.760000e-06    -3.760000e-06   171.595550   
4    4.134435     1.050000e-05    -1.050000e-05   172.979370   

   koi_time0bk_err1  ...  koi_steff_err2  koi_slogg  koi_slogg_err1  \
0          0.003520  ...             -81      4.467           0.064   
1          0.000581  ...            -176      4.544           0.044   
2          0.000115  ...            -174      4.564           0.053   
3          0.001130  ...            -211      4.438           0.070   
4          0.001900  ...            -232      4.486           0.054   

   koi_slogg_err2  koi_srad  koi_srad_err1  koi_srad_err2         ra  \
0          -0.096     0.927          0.105         -0.061  291.93423   
1          -0.176     0.868          0.233         -0.078  297.00482   
2          -0.168     0.791          0.201         -0.067  285.53461   
3          -0.210     1.046          0.334         -0.133  288.75488   
4          -0.229     0.972          0.315         -0.105  296.28613   

         dec  koi_kepmag  
0  48.141651      15.347  
1  48.134129      15.436  
2  48.285210      15.597  
3  48.226200      15.509  
4  48.224670      15.714  

[5 rows x 41 columns]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [11]:
X = df.drop('koi_disposition', axis = 1)
X.head()
y = df['koi_disposition']


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = 0.80)

#A second split for a Hyperparameter tuning set
X_train, X_hp_train, y_train, y_hp_train = train_test_split(X_train, y_train, random_state=0, train_size = 0.5)

# Train the Model



In [12]:


rf_model = RandomForestClassifier(n_estimators=200, random_state = 0, max_features = 'auto')
rf_model = rf_model.fit(X_train,y_train)

print(f"Training Data Score: {rf_model.score(X_train, y_train)}")
print(f"Testing Data Score: {rf_model.score(X_test, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.8969957081545065


In [27]:
param_grid = {
    'max_depth': [17, 20, 22],
    'min_samples_leaf': [2,3,4],
    'min_samples_split': [1,2,3],
    'n_estimators': [275, 300, 325]
}
grid = GridSearchCV(rf_model, param_grid, verbose=3, cv = 7, n_jobs = -3)
grid.fit(X_hp_train, y_hp_train)

Fitting 7 folds for each of 81 candidates, totalling 567 fits
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-3)]: Done  28 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-3)]: Done 134 tasks      | elapsed:   52.3s
[Parallel(n_jobs=-3)]: Done 294 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-3)]: Done 518 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-3)]: Done 567 out of 567 | elapsed:  4.9min finished


GridSearchCV(cv=7,
             estimator=RandomForestClassifier(max_depth=20, min_samples_leaf=3,
                                              n_estimators=300,
                                              random_state=0),
             n_jobs=-3,
             param_grid={'max_depth': [17, 20, 22],
                         'min_samples_leaf': [2, 3, 4],
                         'min_samples_split': [1, 2, 3],
                         'n_estimators': [275, 300, 325]},
             verbose=3)

In [28]:
print(grid.best_params_)
print(grid.best_score_)

{'max_depth': 20, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 300}
0.8848451485857503


In [31]:
rf_model = RandomForestClassifier(n_estimators=300,max_depth = 20, min_samples_leaf = 3, min_samples_split = 2, random_state = 0)
rf_model = rf_model.fit(X_train,y_train)

print(f"Training Data Score: {rf_model.score(X_train, y_train)}")
print(f"Testing Data Score: {rf_model.score(X_test, y_test)}")

Training Data Score: 0.9774678111587983
Testing Data Score: 0.894134477825465


In [30]:
import pickle

with open('pickle_jar/rf_model.pickle', 'wb') as dill:
    pickle.dump(rf_model,dill)